In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [40]:
df =  pd.read_excel("Telco_customer_churn.xlsx")

In [41]:
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [43]:
#Cambiar el tipo de dato de total churn de objeto a númerico
df["Total Charges"] = pd.to_numeric(df["Total Charges"], errors="coerce")

In [44]:
#Obteniendo el número de columnas de cada tipo
print(f"""Número de columnas de:
tipo coma flotante: {len(df.select_dtypes("float").columns)}
tipo entero: {len(df.select_dtypes('int').columns)}
tipo objeto (cadena de carateres): {len(df.select_dtypes('object').columns)}
""")



Número de columnas de:
tipo coma flotante: 4
tipo entero: 6
tipo objeto (cadena de carateres): 23



In [45]:
#Número de nulos por columna
data_na = df.isnull().sum()/len(df)
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({"mean_nulls (%)": data_na*100,
                             "number_nulls": df[data_na.index].isna().sum()})
missing_data

,mean_nulls (%),number_nulls
Churn Reason,73.463013,5174
Total Charges,0.156183,11


In [46]:
#Determinando por qué Churn Reason tiene tanto nulos
df["Churn Label"].value_counts(normalize=True)

Churn Label
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

El porcentaje de personas que no tienen "Churn Label" es igual al porcentaje de personas que no tienen "Churn Reason".
Esta columna podría ser eliminada, ya que por regla general se eliminan las columnas con una presencia mayor al 50%

In [47]:
#Nulos de la columna "Total Charges"
df[df['Total Charges'].isna()]

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
2234,4472-LVYGI,1,United States,California,San Bernardino,92408,"34.084909, -117.258107",34.084909,-117.258107,Female,...,Two year,Yes,Bank transfer (automatic),52.55,NaN,No,0,36,2578,NaN
2438,3115-CZMZD,1,United States,California,Independence,93526,"36.869584, -118.189241",36.869584,-118.189241,Male,...,Two year,No,Mailed check,20.25,NaN,No,0,68,5504,NaN
2568,5709-LVOEQ,1,United States,California,San Mateo,94401,"37.590421, -122.306467",37.590421,-122.306467,Female,...,Two year,No,Mailed check,80.85,NaN,No,0,45,2048,NaN
2667,4367-NUYAO,1,United States,California,Cupertino,95014,"37.306612, -122.080621",37.306612,-122.080621,Male,...,Two year,No,Mailed check,25.75,NaN,No,0,48,4950,NaN
2856,1371-DWPAZ,1,United States,California,Redcrest,95569,"40.363446, -123.835041",40.363446,-123.835041,Female,...,Two year,No,Credit card (automatic),56.05,NaN,No,0,30,4740,NaN
4331,7644-OMVMY,1,United States,California,Los Angeles,90029,"34.089953, -118.294824",34.089953,-118.294824,Male,...,Two year,No,Mailed check,19.85,NaN,No,0,53,2019,NaN
4687,3213-VVOLG,1,United States,California,Sun City,92585,"33.739412, -117.173334",33.739412,-117.173334,Male,...,Two year,No,Mailed check,25.35,NaN,No,0,49,2299,NaN
5104,2520-SGTTA,1,United States,California,Ben Lomond,95005,"37.078873, -122.090386",37.078873,-122.090386,Female,...,Two year,No,Mailed check,20.00,NaN,No,0,27,3763,NaN
5719,2923-ARZLG,1,United States,California,La Verne,91750,"34.144703, -117.770299",34.144703,-117.770299,Male,...,One year,Yes,Mailed check,19.70,NaN,No,0,69,4890,NaN
6772,4075-WKNIU,1,United States,California,Bell,90201,"33.970343, -118.171368",33.970343,-118.171368,Female,...,Two year,No,Mailed check,73.35,NaN,No,0,44,2342,NaN


La columna "Total Charges" es el resultado de multiplicar el número de meses que el cliente uso el servicio "Tenure Months" por los cargos mensuales "Monthly Charges"

In [48]:
df["Total Charges"].fillna(df["Monthly Charges"] * df["Tenure Months"], inplace = True)

### **El problema de baja y alta cardinalidad**

* **Baja cardinalidad**: Una columna que posee un solo tipo de valor es considerada como una columna uniforme y representa un problema para el análisis de datos o aprendizaje automático, ya que no provee información.

* **Alta cardinalidad**: Cuando cada fila posee un unico valor representa un desafío para el modelado, puedo conducir a un inmenso número de categorías únicas, que hace que trabajar con los datos sea más dificultoso.

In [49]:
df.nunique().sort_values()

Count                   1
Country                 1
State                   1
Senior Citizen          2
Churn Value             2
Churn Label             2
Paperless Billing       2
Gender                  2
Partner                 2
Dependents              2
Phone Service           2
Online Backup           3
Online Security         3
Streaming TV            3
Multiple Lines          3
Internet Service        3
Device Protection       3
Contract                3
Streaming Movies        3
Tech Support            3
Payment Method          4
Churn Reason           20
Tenure Months          73
Churn Score            85
City                 1129
Monthly Charges      1585
Longitude            1651
Latitude             1652
Lat Long             1652
Zip Code             1652
CLTV                 3438
Total Charges        6531
CustomerID           7043
dtype: int64

**Columanas con problemas de cardinalidad**

*Columnas con baja cardinalidad:*

* Count column
* Country column
* State column

*Columnas con alta cardinalidad:*

* Customer ID column
* Zip Code column

In [50]:
df.groupby(["Country", "State"]).size()

Country        State     
United States  California    7043
dtype: int64